In [20]:
import pandas as pd
import os
import matplotlib
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet101
from keras.models import Model
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers, Input
from keras.optimizers import SGD
from keras.losses import Huber
from keras.optimizers import Adam
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import sys

In [24]:

#Store path
train_dir = '/home/jwf5/venv/FOODX-251_Dataset/train_set/'
val_dir = '/home/jwf5/venv/FOODX-251_Dataset/val_set/'

train_df = pd.read_csv('FOODX-251_Dataset/train_labels.csv')
train_df['img_name'] = train_dir + train_df['img_name'].astype(str)

val_df = pd.read_csv('FOODX-251_Dataset/val_labels.csv')
val_df['img_name'] = val_dir + val_df['img_name'].astype(str)
train_df.head()
tLab = train_df['label'].to_numpy()
vLab = val_df['label'].to_numpy()
tLab.astype(np.int64)
vLab.astype(np.int64)
tPath = train_df['img_name'].to_numpy()
vPath = val_df['img_name'].to_numpy()

In [26]:
def print_progress(count, total):
    pct_complete = float(count) / total
    msg = "\r- Progress: {0:.1%}".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def convert(image_paths , labels, out_path):
    # Args:
    # image_paths   List of file-paths for the images.
    # labels        Class-labels for the images.
    # out_path      File-path for the TFRecords output file.

    print("Converting: " + out_path)

    # Number of images. Used when printing the progress.
    num_images = len(image_paths)

    # Open a TFRecordWriter for the output-file.
    with tf.python_io.TFRecordWriter(out_path) as writer:

        # Iterate over all the image-paths and class-labels.
        for i, (path, label) in enumerate(zip(image_paths, labels)):
            # Print the percentage-progress.
            print_progress(count=i, total=num_images-1)

            img_bytes = open(path,'rb').read()

            # Create a dict with the data we want to save in the
            # TFRecords file. You can add more relevant data here.
            data = \
            {
                'image': wrap_bytes(img_bytes),
                'label': wrap_int64(label)
            }

            # Wrap the data as TensorFlow Features.
            feature = tf.train.Features(feature=data)

            # Wrap again as a TensorFlow Example.
            example = tf.train.Example(features=feature)

            # Serialize the data.
            serialized = example.SerializeToString()

            # Write the serialized data to the TFRecords file.
            writer.write(serialized)
convert(tPath, tLab, 'FOODX-251_Dataset/train_TFR' )
convert(vPath, vLab, 'FOODX-251_Dataset/val_TFR' )

Converting: FOODX-251_Dataset/train_TFR
- Progress: 100.0%Converting: FOODX-251_Dataset/val_TFR
- Progress: 100.0%

In [8]:
def save(dataset, location='FOODX-251_Dataset/'):
#     dataset = dataset.map(tf.io.serialize_tensor)
    writer = tf.data.experimental.TFRecordWriter(location)
    writer.write(dataset)
    return location
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label
with tf.python_io.TFRecordWriter("csv.tfrecords") as writer:
    # A vector of filenames.
    filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg"])

    # `labels[i]` is the label for the image in `filenames[i].
    labels = tf.constant([0, 37])

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(_parse_function)
    save(dataset)

TypeError: `dataset` must produce scalar `DT_STRING` tensors whereas it produces shape (TensorShape([Dimension(28), Dimension(28), Dimension(None)]), TensorShape([])) and types (tf.float32, tf.int32)